# Data Exploration
### Inspecting our features

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Playlists: 'Beast Mode', 'New Music Friday', 'Mood Booster', 'Deep Focus', 'Songs to Sing in the Shower', 'Comfort Zone', 'No Playlist'


In [ ]:
#df = pd.read_pickle("dataframe.pkl")
df = pd.read_pickle("dataframe_w_labels.pkl")

In [ ]:
#print(df['PlaylistTitle'].unique())
#print(len(df))
df['OnPlaylist'] = df['PlaylistTitle'].apply(lambda x: 0 if x == "No Playlist" else 1)
pd.set_option("display.max_rows", None, "display.max_columns", None) # Displays the entire dataframe in nootebook, not just a preview
#df

In [ ]:
#Get rid of mister Bach
df = df[df['MainArtist'] != 'Johann Sebastian Bach']
df = df[df.index != 247]
df = df[df.index != 182]
#df[df['MainArtist'].isna() == False]
print(len(df))

# Check for duplicated
df.drop_duplicates(subset='TrackID', keep="first", inplace=True)
print(len(df.groupby("TrackID").count()["index"])) 

## See how many of each playlist
## 73 of each
print(len(df.groupby("PlaylistTitle").count()["index"])) 

## Only keep 73 of each
#df = df.groupby('PlaylistTitle').apply(lambda x: x.sample(73))




In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df = df.where(df.OnPlaylist==1)

print(len(df))



In [ ]:
## Looking at Discovery versus Mood playlists
df_disc = df.where((df.PlaylistTitle == "New Music Friday"))

df_disc = df_disc[['PlaylistTitle','TrackID','NoFeaturing', 'danceability', 'energy', 'key', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms',
       'time_signature']].dropna()

## Moods
df_over = df.where(df.PlaylistTitle != "Songs to Sing in the Shower")
#df_over = df_over.where(df.PlaylistTitle != "Songs to Sing in the Shower")
df_over = df_over[['PlaylistTitle','TrackID','NoFeaturing', 'danceability', 'energy', 'key', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms',
       'time_signature']].dropna()

## Scaling
df_disc[['NoFeaturing','time_signature','key','tempo', 'duration_ms', 'loudness']] = scaler.fit_transform(df_disc[['NoFeaturing','time_signature','key','tempo', 'duration_ms', 'loudness']])

## Scaling
df_over[['NoFeaturing','time_signature','key','tempo', 'duration_ms', 'loudness']] = scaler.fit_transform(df_over[['NoFeaturing','time_signature','key','tempo', 'duration_ms', 'loudness']])


#print(df_over.columns)
#df_over
fig = plt.figure(figsize=(13,3))
fig = sns.boxplot(data=df_over)
fig = sns.stripplot(data = df_over, alpha=0.5)
plt.xticks(rotation=45)
plt.savefig("boxplot.pdf", bbox_inches = 'tight')

fig2 = plt.figure(figsize=(13,3))
fig2 = sns.boxplot(data=df_disc)
fig2 = sns.stripplot(data=df_disc, alpha=0.5)
plt.xticks(rotation=45)

In [ ]:
df_delta = abs(df_disc.describe() - df_over.describe())
df_delta = df_delta.transpose()
df_delta["var"] = df_delta["std"]*df_delta["std"]
df_delta.sort_values(by="std", ascending=False)


In [ ]:
df_var = df_over.var()
df_var = df_var.sort_values(ascending=False)
#print(df_var)

fig = plt.figure(figsize=(11,2))
fig = df_var.plot.bar()
fig.set_ylim(0,0.20)
plt.xticks(rotation=45)
#df_over[['NoFeaturing', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
#       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms',
#       'time_signature']]
plt.savefig("variance_explore.pdf", bbox_inches = 'tight')

## Discovery
df_var = df_disc.var()
df_var = df_var.sort_values(ascending=False)
#print(df_var)

fig2 = plt.figure(figsize=(13,2))
fig2 = df_var.plot.bar()
fig2.set_ylim(0,0.20)
plt.xticks(rotation=45)
#df_over[['NoFeaturing', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
#       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms',
#       'time_signature']]

In [ ]:
# Reset to include playlist name
df_over = df.where(df.PlaylistTitle != "Songs to Sing in the Shower")
#df_over = df_over.where(df.PlaylistTitle != "Songs to Sing in the Shower")
df_over = df_over[['PlaylistTitle','PlaylistID', 'NoFeaturing', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms',
       'time_signature']].dropna()

## Scaling
df_over[['NoFeaturing','time_signature','key','tempo', 'duration_ms', 'loudness']] = scaler.fit_transform(df_over[['NoFeaturing','time_signature','key','tempo', 'duration_ms', 'loudness']])

## Same order as above
sort_order = [ "instrumentalness", "acousticness", "key", "energy", "valence", "NoFeaturing", "duration_ms", "loudness", "tempo", "danceability", "liveness", "time_signature", "speechiness"]

df_var2 = df_over.groupby("PlaylistTitle").var()
df_var2 = df_var2[["mode", "instrumentalness", "acousticness", "key", "energy", "valence", "NoFeaturing", "duration_ms", "loudness", "tempo", "danceability", "liveness", "time_signature", "speechiness"]]
df_var2 = df_var2.transpose()
#print(df_var2)


fig = plt.figure(figsize=(13,3))
fig = df_var2.plot.bar()
plt.xticks(rotation=45)
#df_over[['NoFeaturing', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
#       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms',
#       'time_signature']]

In [ ]:
#Exlude Deep Focus
pls = ["Deep Focus", "Beast Mode", "Mood Booster", "Comfort Zone", "New Music Friday"]

#for i in pls:

#	df['OnPlaylist'] = df['PlaylistTitle'].apply(lambda x: 1 if x == i else 0)
#	g = sns.PairGrid(df[['OnPlaylist','key', 'mode','tempo', 'duration_ms', 'loudness']], hue="OnPlaylist")
#	g.fig.suptitle(i)
#	g.map_diag(sns.histplot)
#	g.map_offdiag(sns.scatterplot, alpha=0.6)
#	g.add_legend()

#g = sns.PairGrid(df[['PlaylistTitle','NoFeaturing','tempo', 'duration_ms', 'loudness']], hue="PlaylistTitle")
#g.fig.suptitle("All")
#g.map_diag(sns.histplot)
#g.map_offdiag(sns.scatterplot, alpha=0.2)
#g.add_legend()

In [ ]:

features_ls = ["NoFeaturing", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]
pls = ["Deep Focus", "Beast Mode", "Mood Booster", "Comfort Zone", "New Music Friday"]

## DF for this block
dff = df_over
for i in sort_order:
    
    fig, ax =plt.subplots(1,5, sharex='col', sharey='row')
    fig.set_size_inches(8, 1.5)
    counter = 0
    ax[counter].set_title(i)
    for j in pls:
        
        dff['OnPlaylist'] = dff['PlaylistTitle'].apply(lambda x: 1 if x == j else 0)
        ax[counter].grid(False)
        
       
        sns.distplot(dff.where(dff.OnPlaylist==1)[i],ax=ax[counter], color="darkblue", label=j)
        sns.distplot(dff[i],ax=ax[counter], color="darkgreen", label="All")
        ax[counter].set_xlabel(j)
        

        counter = counter + 1
    ax[4].legend(['Playlist x', 'Average'])
    plt.savefig("0_"+i+".pdf",bbox_inches = 'tight')





In [ ]:
## Average

overview = pd.DataFrame()
for i in pls:
    tmp = df[["instrumentalness", "energy", "danceability", "valence", "NoFeaturing", "loudness", "tempo", "duration_ms", "key", "mode"]].where(df_over.PlaylistTitle==i).describe().transpose()["mean"]
    overview[i]=round(tmp,1)

overview

In [ ]:
## Final features ["NoFeaturing", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms"]
## Final predicter ["PlaylistTitle"]

#DROP on playlist
df_over = df_over.drop(columns="OnPlaylist")

## Therefore, these are dropped: ['index', 'PlaylistID',, 'TrackID', 'MainArtist',
                                # 'id', 'uri', 'track_href', 'analysis_url',
                                # 'time_signature', 'Artist', 'Label', 'BigLabel']

df_tmp = df_over [["NoFeaturing", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms","PlaylistTitle"]].dropna()
X = df_tmp[["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms"]]
y = df_tmp[["PlaylistTitle"]]
print(len(X), len(y))

## For disc
df_tmp2 = df_disc [["NoFeaturing", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms","PlaylistTitle"]].dropna()
# df_tmp2 = df_tmp2.append(df_tmp)
X2 = df_tmp2[["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms"]]
y2 = df_tmp2[["PlaylistTitle"]]
print(len(X2), len(y2))






In [ ]:
#Equal amount of each playlist


### T-SNES

Using T-SNES to try and plot the features.


In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D


## PCA mood to comparison
pca = PCA(n_components=3)
pca_result = pca.fit_transform(X)
X['pca-one'] = pca_result[:,0]
X['pca-two'] = pca_result[:,1] 
X['pca-three'] = pca_result[:,2]

## y to digits
#print(y)
y["PlaylistTitle"] = y["PlaylistTitle"].astype('category')
y["PlaylistTitle"] = y["PlaylistTitle"].cat.codes
X['y'] = y["PlaylistTitle"]
#print(y)

## PCA disc to comparison
pca = PCA(n_components=3)
pca_result = pca.fit_transform(X2)
X2['pca-one'] = pca_result[:,0]
X2['pca-two'] = pca_result[:,1] 
X2['pca-three'] = pca_result[:,2]

## y to digits
#print(y)
y2["PlaylistTitle"] = y2["PlaylistTitle"].astype('category')
y2["PlaylistTitle"] = y2["PlaylistTitle"].cat.codes
X2['y'] = y2["PlaylistTitle"]








In [ ]:
## Correlation of features and label
df_tmp["PlaylistTitle"] = df_tmp["PlaylistTitle"].astype('category')
df_tmp["y"] = df_tmp["PlaylistTitle"].cat.codes
df_tmp2['y'] = X2['y']
df_corr_all = df_tmp
df_corr_disc = df_tmp2

# Making label a categorical
df_over["PlaylistTitle"] = df_over["PlaylistTitle"].astype('category')
df_over["y"] = df_over["PlaylistTitle"].cat.codes
#print(df_corr_all.tail(5))

#correlation matrix for all
print(df_over.columns)
corrmat = df_over.where(df_over.y!=4).corr()[['y']][:-1].sort_values("y")
corrmat["y"] = round(corrmat["y"],1)

f, ax = plt.subplots(figsize=(8, 0.5))
labels = pd.DataFrame([[corrmat.iloc[0].values,"","","","","","","","","","","","",corrmat.iloc[13].values]])
#print(labels.shape)
#print(corrmat.shape)
sns.heatmap(corrmat.transpose(),  annot=labels.values ,fmt="",vmin=-0.5, vmax=0.5,ax=ax, yticklabels = [""], cbar=False);
ax.axvline(x=0,linewidth=1, color="w")
ax.axvline(x=1,linewidth=1, color="w")
ax.axvline(x=2,linewidth=1, color="w")
ax.axvline(x=3,linewidth=1, color="w")
ax.axvline(x=4,linewidth=1, color="w")
ax.axvline(x=5,linewidth=1, color="w")
ax.axvline(x=6,linewidth=1, color="w")
ax.axvline(x=7,linewidth=1, color="w")
ax.axvline(x=8,linewidth=1, color="w")
ax.axvline(x=9,linewidth=1, color="w")
ax.axvline(x=10,linewidth=1, color="w")
ax.axvline(x=11,linewidth=1, color="w")
ax.axvline(x=12,linewidth=1, color="w")
ax.axvline(x=13,linewidth=1, color="w")
plt.savefig("feature_importance.pdf",bbox_inches = 'tight')

print(df_corr_all.where(df_corr_all.y!=4).corr())

""" fig, ax =plt.subplots(1,3)
fig.set_size_inches(12, 9)

## Heat map for all
corrmat = df_corr_all.corr()
ax[0].set_title("Aggregated")
sns.heatmap(corrmat, vmax=.8, square=True);

## heat map for disc
corrmat = df_corr_disc.corr()
ax[1].set_title("Discovery")
sns.heatmap(corrmat, vmax=.8, square=True);

## Heat map for Mood
corrmat = df_corr_all.where(df_corr_all.y!=4).corr()
ax[2].set_title("Mood")
sns.heatmap(corrmat, vmax=.8, square=True); """

f,(ax1,ax2,ax3, axcb) = plt.subplots(1,4, 
            gridspec_kw={'width_ratios':[1,1,1,0.1]})
ax1.get_shared_y_axes().join(ax2,ax3)
g1 = sns.heatmap(df_over.corr()[:-1],cbar=False,ax=ax1)
g1.set_title("Aggregated")
g1.set_ylabel('')
g1.set_xlabel('')
g2 = sns.heatmap(df_over.where(df_corr_all.y==4).corr(),cbar=False,ax=ax2)
g2.set_title("Discovery")
g2.set_ylabel('')
g2.set_xlabel('')
g2.set_yticks([])
g3 = sns.heatmap(df_over.where(df_corr_all.y!=4).corr()[:-1],ax=ax3, cbar_ax=axcb)
g3.set_title("Mood")
g3.set_ylabel('')
g3.set_xlabel('')
g3.set_yticks([])

# may be needed to rotate the ticklabels correctly:
for ax in [g1,g2,g3]:
    ax.set_xticklabels([])
    tly = ax.get_yticklabels()
    ax.set_yticklabels(tly, rotation=0)

plt.savefig("heatmap.pdf",bbox_inches = 'tight')
plt.show()




